In [1]:
!pip install nibabel tqdm scikit-image
#리눅스 사용 시 !을 쓴다.

In [2]:
import os
import sys
import random
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from scipy.ndimage import zoom

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.autograd import Variable

In [3]:
#구글 드라이브를 쓰자
from google.colab import drive
#루트를 정해준다.
ROOT = "/content/drive"
print(ROOT)

drive.mount(ROOT) #we mount the google drive at /content/drive

#import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join


MY_GOOGLE_DRIVE_PATH = '/content/drive/My Drive/Colab Notebooks'

print("MY_GOOGLE_DRIVE_PATH: ", MY_GOOGLE_DRIVE_PATH)

%cd "{MY_GOOGLE_DRIVE_PATH}"


/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MY_GOOGLE_DRIVE_PATH:  /content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks


In [4]:
#모델 작성하는 Pytoch에서 class는 모듈이다.
#SRNN 클래스 모델 선언 
class SRCNN(nn.Module):
  def __init__(self): #모델이 어떤 layer을 쓰는냐
    super(SRCNN, self).__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(1,64,9,padding = 4), #input채널,output채널, 필터 크기, 패딩 지정
        nn.ReLU(),
        nn.Conv2d(64,32,1), #convolution 사이즈가 1이기에 패딩 필요없다.
        nn.ReLU(),
        nn.Conv2d(32,1,5,padding =2) #논문에 다 나온다.
    )
    #하나의 x가 들어오면 init에서 정의한 것을 layer을 통과하는 방식으로 함수를 만든다
    def forward(self, x):  
      return self.layers(x)


In [5]:
def preprocessing(image_name_raw):
  #nii파일을 nibabel으로 불러온다.
  raw_image = nib.load(image_name_raw)

  #get_fdata 다른 헤더정보도 가져온다.(여러정보)
  raw_array = np.array(raw_image.get_fdata(), dtype=np.float32)

  #전처리 99.9%보다 복셀들을 사용하도록하고 이상인것은 그냥 전기값으로 설정
  clip_value = np.sort(raw_array.ravel())[int(np.prod(raw_array.shape)*0.999)]
  clipped_image = np.clip(raw_array, 0, clip_value)

  #shape를 가져와 이미지 크기를 맞춰준다. 
  H, W, D = clipped_image.shape
  cropped_image = clipped_image[:H - H%3, :W - W%3, : D - D%3]
  image = cropped_image/ cropped_image.max()

  return image

In [6]:
#다운 샘플링을 가지고 원본이미지에 가깝게 만들어준다.
def get_lr(image) :
  #1/3만큼 크기로 만든다..
  downscaled_lr = zoom(image, 1/3.0, order = 2, prefilter= False)

  #3만큼 올린다.
  lr = np.clip(zoom(downscaled_lr, 3, order = 2, prefilter=False), 0, image.max())
  return lr


In [7]:
#dataset 클래스 정리
class BrainDataset(DataLoader):
  def __init__(self):
    #이미지를 불러와서 데이터 셋으로 만들겠다.
    self.dataset_dir = os.path.join(MY_GOOGLE_DRIVE_PATH, 'srcnn', 'train_data')
    
    #list 로 만듬
    self.files_list = os.listdir(dataset_dir)

    #메모리에 lr과 hr 저장해오는것을 가져오기 위한 변수들.
    self.image_hr_list = []
    self.image_lr_list = []
    
#   for file_name in tqdm(self.files_list):
    for file_name in self.files_list:
      print(file_name)

      #데이터 셋에 있는 file_name을 가져온다
      image_name_raw = os.path.join(self.dataset_dir, file_name)

      #전처리
      image = preprocessing(image_name_raw)
      H, W, D = image.shape

      for i in range(H):
        #까만 이미지만 아니면 통과
        if np.any(image[i]):
          lr = get_lr(image[i])

          #배치 사이즈를 제일 앞에 두고 그다음 채널수를 앞에 두는데
          #채널이 중간에 엄청 많으니 새로 만드는 작업을 한다.
          self.image_hr_list.append(image[i,np.newaxis, :, :])
          self.image_lr_list.append(lr[np.newaxis, :, :])

  def __getitem__(self, index): #몇번째 데이터를 리턴해라이다.
    return self.image_lr_list[index], self.image_hr_list[index]
  def __len__(self): #몇번째 index가 있느냐
    return len(self.image_hr_list)



In [99]:
#dataset을 표시해준다.
dataset_dir = os.path.join(MY_GOOGLE_DRIVE_PATH, 'srcnn', 'train_data')
print(os.listdir(dataset_dir))

/content/drive/My Drive/Colab Notebooks/srcnn/train_data
['T1w_acpc_dc_restore_brain_100307.nii.gz', 'T1w_acpc_dc_restore_brain_100206.nii.gz', 'T1w_acpc_dc_restore_brain_101915.nii.gz', 'T1w_acpc_dc_restore_brain_100408.nii.gz', 'T1w_acpc_dc_restore_brain_102008.nii.gz', 'T1w_acpc_dc_restore_brain_101410.nii.gz', 'T1w_acpc_dc_restore_brain_102311.nii.gz', 'T1w_acpc_dc_restore_brain_100610.nii.gz', 'T1w_acpc_dc_restore_brain_101309.nii.gz', 'T1w_acpc_dc_restore_brain_101107.nii.gz', 'T1w_acpc_dc_restore_brain_101006.nii.gz']


In [100]:
#cuda 0번 GPU쓴다
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#cuda가 있는지 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {}'.format(device))

Device: cuda


In [101]:
BATCH_SIZE = 16

In [104]:
#데이터 셋을 가져오자
dataset = BrainDataset()

#데이터 셋을 기반으로 읽어드는 방식
#shuffle 은 한번 train 할때마다 섞어서 다시 학습
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)


  0%|          | 0/11 [00:00<?, ?it/s]

/content/drive/My Drive/Colab Notebooks/srcnn/train_data



100%|██████████| 11/11 [00:27<00:00,  2.52s/it]


In [105]:
#모델을 cuda로 보냄
model = SRCNN()
model.to(device)

SRCNN(
  (layers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (1): ReLU()
    (2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
)

In [106]:
#Loss와 Optimizer을 가져온다Loss와 Optimizer을 가져온다
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001) #모델의 파라메타 업데이트 lr을 0.001로 설정

In [107]:
#모델 트레이닝
for epoch in range(10):
  print('epoch' + str(epoch +1))

  #데이터 로더에 있는 image_lr과 images_hr을 가져와 cuda에 넣는다.
  
  for images_lr, images_hr in dataloader:
    images_lr = images_lr.to(device)
    images_hr = images_hr.to(device)


    #optimizer 세팅
    optimizer.zero_grad()

    #모델에 입력값을 통과시킨다.
    output = model(images_lr)

    #결과 값을 실제 결과와 비교하여 loss 계산
    loss = criterion(images_hr, output)

    #backward로 loss 계산
    loss.backward()
    optimizer.step()

    print(loss.detach().cpu().numpy())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.



Epoch 9:   5%|▌         | 7/135 [00:00<00:15,  8.49it/s, loss: 0.0008]








Epoch 9:   6%|▌         | 8/135 [00:00<00:15,  8.35it/s, loss: 0.0008]








Epoch 9:   6%|▌         | 8/135 [00:01<00:15,  8.35it/s, loss: 0.0008]








Epoch 9:   7%|▋         | 9/135 [00:01<00:14,  8.50it/s, loss: 0.0008]








Epoch 9:   7%|▋         | 9/135 [00:01<00:14,  8.50it/s, loss: 0.0008]








Epoch 9:   7%|▋         | 10/135 [00:01<00:14,  8.45it/s, loss: 0.0008]








Epoch 9:   7%|▋         | 10/135 [00:01<00:14,  8.45it/s, loss: 0.0008]








Epoch 9:   8%|▊         | 11/135 [00:01<00:15,  8.26it/s, loss: 0.0008]








Epoch 9:   8%|▊         | 11/135 [00:01<00:15,  8.26it/s, loss: 0.0008]








Epoch 9:   9%|▉         | 12/135 [00:01<00:14,  8.33it/s, loss: 0.0008]








Epoch 9:   9%|▉         | 12/135 [00:01<00:14,  8.33it/s, loss: 0.0008]








Epoch 9:  10%|▉         | 13/135 [00:01<00:14,  8.46it/s, loss: 0.0008]








Epoch 

In [ ]:
image_name_raw_test = os.path.join('./Tlw_')
image)test = preprocessing(image_name_raw_test)
hr_test_slice = image_test[120]
lr_test_slice = get_lr(hr_test_slice)
plt.imshow(lr_test_slice, cmap = 'gray')


In [ ]:
model.eval()
image_test_tensor = ToTensor()(lr_test_slice)
